In [ ]:
# EVALUATION TASK: Comparing three retrieval methods for top-1500 documents.
# EVALUATION METRIC: Coverage/Intersection or Average Recall for all Queries.
# a. Doing a Dense SimCSE Retrival for 1.5k documents from the base BM25@5K documents.
# b. Doing a direct BM25 retrieval on 1.5k documents only.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1. Building a new index with basic preprocessing to the document corpus.
# installing linux related stuff for pyserini
!sudo apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 40 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 0s (708 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty

In [ ]:
# installing important packages for analyzing the code.
!pip install jsonlines
!pip install pyserini
!pip install faiss

     |████████████████████████████████| 84.6 MB 107 kB/s 
     |████████████████████████████████| 4.0 MB 29.4 MB/s 
     |████████████████████████████████| 13.5 MB 35.0 MB/s 
     |████████████████████████████████| 5.2 MB 33.5 MB/s 
     |████████████████████████████████| 1.2 MB 31.2 MB/s 
     |████████████████████████████████| 2.0 MB 32.0 MB/s 
     |████████████████████████████████| 6.0 MB 32.1 MB/s 
     |████████████████████████████████| 1.1 MB 36.9 MB/s 
     |████████████████████████████████| 188 kB 33.4 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 653 kB 32.6 MB/s 
     |████████████████████████████████| 181 kB 32.5 MB/s 
     |████████████████████████████████| 457 kB 20.0 MB/s 
     |████████████████████████████████| 10.1 MB 32.1 MB/s 
     |████████████████████████████████| 895 kB 32.3 MB/s 
     |████████████████████████████████| 6.6 MB 30.7 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |██████████

In [ ]:
# 2. Doing a SimCSE model retrieval for performing dense retrieval.
!pip install simcse

     |████████████████████████████████| 25.9 MB 10.9 MB/s 
     |████████████████████████████████| 14.8 MB 29.7 MB/s 
  Created wheel for simcse: filename=simcse-0.4-py3-none-any.whl size=15033 sha256=823861ece04a78cd3e50b84d24c032831db666e0320991c1da7ae8cbb77d0dcc
  Stored in directory: /root/.cache/pip/wheels/29/b0/58/448de751ccbe2a0e75c054e34416e7a5a1aead371d5d163450
Successfully built simcse
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires numpy>=1

In [ ]:
# import statements for index building.
import pandas as pd
import jsonlines
import re

In [ ]:
# query expansion corpus to be loaded.
qcorpus_list = list(jsonlines.open('drive/My Drive/touche-2022-prototyping/touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl'))

In [ ]:
# printing demo query for review.
qcorpus_list[0]

{'chatNoirUrl': 'https://chatnoir.eu/cache?uuid=25e04d49-8df7-58c9-8fae-c5bd54a070ba&index=cw12&raw&plain',
 'contents': "Do Asian-Americans Face Bias in Admissions at Elite Colleges? - NYTimes.com Home Page Today's Paper Video Most Popular Times Topics Search All NYTimes.com Education World U.S. Politics Education Bay Area Chicago Texas N.Y. / Region Business Technology Science Health Sports Opinion Arts Style Travel Jobs Real Estate Autos February 8, 2012, 1:43 pm Do Asian-Americans Face Bias in Admissions at Elite Colleges? By DANIEL E. SLOTNIK 6:08 p.m. | Updated A statement from Princeton was added to the story. The Department of Education’s Office for Civil Rights is examining complaints thatHarvard and Princeton have discriminated against Asian-American undergraduate applicants, highlighting a concern of many Asian-American parents. The inquiry was first reported by Bloomberg News.<query> do asian americans face bias in admissions</query>",
 'id': 'clueweb12-0000tw-00-14115___1'

In [ ]:
# making respective for creating jsonl file and respective index file.
!mkdir sample_collection_jsonl
!mkdir /content/drive/MyDrive/touche-2022-prototyping/indexes
!mkdir /content/drive/MyDrive/touche-2022-prototyping/indexes/baseline_index

mkdir: cannot create directory ‘sample_collection_jsonl’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/touche-2022-prototyping/indexes’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/touche-2022-prototyping/indexes/baseline_index’: File exists


In [ ]:
# removing the extra directory
!rm -r  sample_data

In [ ]:
# all the document processing related import statements.
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# loading all the stop words frome english language.
stop_words = set(stopwords.words("english"))

In [ ]:
# storing all documents in the pyserini compatible format.
output = jsonlines.open('sample_collection_jsonl/documents.jsonl', 'w')
for vals in qcorpus_list:
    document = vals['contents']
    document = document.lower()
    # Remove punctuation and numbers
    document = re.sub(r"[^A-Za-z\s]+", " ", document)
    # Remove extra XML query tag
    document = document.replace("query"," ")
    # Remove extra white spaces
    document = re.sub(r"[\s]{2,}", " ", document)
    # Tokenize string
    word_tokens = word_tokenize(document)
    # Remove stop words
    word_tokens = [
        token for token in word_tokens if token not in stop_words]
    word_tokens = [
        token for token in word_tokens if len(token) > 1]
    # Combining word tokens for results.
    results = " ".join(word_tokens)
    document = results.strip()
    output.write({
        'id': vals['id'],
        'contents': document
    })

In [ ]:
# printing the output of the index.
!head -20 sample_collection_jsonl/documents.jsonl
# below command for building the new baseline index.

{"id": "clueweb12-0000tw-00-14115___1", "contents": "asian americans face bias admissions elite colleges nytimes com home page today paper video popular times topics search nytimes com education world politics education bay area chicago texas region business technology science health sports opinion arts style travel jobs real estate autos february pm asian americans face bias admissions elite colleges daniel slotnik updated statement princeton added story department education office civil rights examining complaints thatharvard princeton discriminated asian american undergraduate applicants highlighting concern many asian american parents inquiry first reported bloomberg news asian americans face bias admissions"}
{"id": "clueweb12-0000tw-00-14115___10", "contents": "insisting upholding affirmative action college admissions ensure america continues extremely race sensitive society everything based race asian americans say take heart competition makes stronger cares ivy league mission n

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /content/sample_collection_jsonl \
  --index /content/drive/MyDrive/touche-2022-prototyping/indexes/baseline_index \
  --generator DefaultLuceneDocumentGenerator \
  --threads 4 \
  --storePositions --storeDocvectors --storeRaw

2022-04-25 18:30:21,751 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Setting log level to INFO
2022-04-25 18:30:21,754 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2022-04-25 18:30:21,754 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2022-04-25 18:30:21,754 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: /content/sample_collection_jsonl
2022-04-25 18:30:21,754 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2022-04-25 18:30:21,755 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2022-04-25 18:30:21,757 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 4
2022-04-25 18:30:21,757 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Language: en
2022-04-25 18:30:21,757 INFO  [main] index.IndexC

In [ ]:
!ls /content/drive/MyDrive/touche-2022-prototyping/indexes/baseline_index/

_2.fdm	_2_Lucene80_0.dvd  _2_Lucene84_0.tim  _2.nvm  _2.tvx
_2.fdt	_2_Lucene80_0.dvm  _2_Lucene84_0.tip  _2.si   segments_3
_2.fdx	_2_Lucene84_0.doc  _2_Lucene84_0.tmd  _2.tvd  write.lock
_2.fnm	_2_Lucene84_0.pos  _2.nvd	      _2.tvm


In [ ]:
# Testing out a demo run for the program.
from pyserini.search.lucene import LuceneSearcher
# searcher = LuceneSearcher('indexes/sample_collection_jsonl')
searcher = LuceneSearcher('/content/drive/MyDrive/touche-2022-prototyping/indexes/baseline_index')
hits = searcher.search('Who is stronger, Hulk or Superman?', k=12)
# Value of K determines the number of documents that can be returned.
print('Number of document matches from the index: '+str(len(hits)))
# Attempted tuning BM25 and RM3, nothing has changed the document ranking below.
for i in range(len(hits)):
    print(f'{i+1} {hits[i].docid} {hits[i].score:.5f}')

Number of document matches from the index: 12
1 clueweb12-1400wb-06-30596___3 15.89310
2 clueweb12-0605wb-29-30060___11 15.78460
3 clueweb12-0605wb-29-30060___10 14.74760
4 clueweb12-1709wb-12-07226___7 14.71700
5 clueweb12-0904wb-33-14589___7 14.31600
6 clueweb12-0605wb-29-30060___2 14.31120
7 clueweb12-1400wb-06-30596___4 14.29860
8 clueweb12-1411wb-25-14634___8 13.85740
9 clueweb12-1014wb-84-07457___12 13.82970
10 clueweb12-0306wb-63-29079___5 13.69080
11 clueweb12-0605wb-29-30060___3 13.41380
12 clueweb12-1400tw-67-08199___28 13.35860


In [ ]:
# Downloading the model beforehand for similarity comparison.
from simcse import SimCSE
model = SimCSE("princeton-nlp/sup-simcse-roberta-large")

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/664 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

In [ ]:
# general import statements for the cosine similarity measurements.
import torch
from simcse import SimCSE
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

In [ ]:
# Import our models for cosine similarity measurements.
from simcse import SimCSE
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-large-uncased")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-large-uncased")

Downloading:   0%|          | 0.00/253 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/621 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

In [ ]:
# Basic testing for the input and output sentences.
texts = [
    "There's a kid on a skateboard.",
    "A kid is skateboarding.",
    "A kid is inside the house."
]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
# Get the embeddings for the input tensors.
with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output
cosine_sim_0_1 = 1 - cosine(embeddings[0], embeddings[1])
print(cosine_sim_0_1)
cosine_sim_0_2 = 1 - cosine(embeddings[0], embeddings[2])
print(cosine_sim_0_2)

0.9810283184051514
0.37127867341041565


In [ ]:
# 3. Doing an initial retrieval of 5K and 1.5K document retrieval. 
# Loading the query topics and jsonl dataset document.
import pandas as pd
touche_topics = pd.read_csv('/content/drive/MyDrive/touche-2022-prototyping/touche_topics_query_expansion.csv')
touche_topics.head()

,number,title,query_base,query_noun,query_synonym1,query_synonym2,query_synonym3,query_antonym1,query_antonym2
0,1,What is the difference between sex and love?,difference sex and love,difference sex love,different difference sex love,unlike difference sex love,dissimilar difference sex love,bad difference sex love,worse difference sex love
1,2,"Which is better, a laptop or a desktop?",better laptop or desktop,laptop desktop,good laptop desktop,well laptop desktop,improve laptop desktop,worse laptop desktop,different laptop desktop
2,3,"Which is better, Canon or Nikon?",better Canon or Nikon,Canon Nikon,good Canon Nikon,well Canon Nikon,improve Canon Nikon,worse Canon Nikon,different Canon Nikon
3,4,What are the best dish detergents?,best dish detergents,detergents,better detergents,good detergents,well detergents,worst detergents,worse detergents
4,5,What are the best cities to live in?,best cities live,cities live,better cities,good cities,well cities,worst cities,worse cities


In [ ]:
# basic input query list
input_queries = list(touche_topics['title'])
input_numbers = list(touche_topics['number'])
input_queries = input_queries[:3]
input_numbers = input_numbers[:3]
print(input_queries[:3])
print(input_numbers[:3])

['What is the difference between sex and love?', 'Which is better, a laptop or a desktop?', 'Which is better, Canon or Nikon?']
[1, 2, 3]


In [ ]:
# getting intial 5k documents as retrieval from the LuceneSearcher BM25 index.
from pyserini.search.lucene import LuceneSearcher
searcher_opt = LuceneSearcher('/content/drive/MyDrive/touche-2022-prototyping/indexes/baseline_index')
# using the optimized parameters that gives the best results.
searcher_opt.set_bm25(1.2, 0.68)

In [ ]:
solution_dict = {}
for id_, q_ in zip(touche_topics['number'], input_queries):
    hits = searcher_opt.search(q_.strip(), k=2000)
    d_list = []
    for h_ in hits:
        # d_= h_.docid.split('___')[0]
        d_= h_.docid
        if d_ not in d_list:
            d_list.append(d_)
    solution_dict[id_] = d_list

In [ ]:
import jsonlines

In [ ]:
# loading the document corpus into the dictionary for iterating over.
qcorpus_list = list(jsonlines.open('drive/My Drive/touche-2022-prototyping/touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl'))

In [ ]:
from string import punctuation

In [ ]:
def preprocess_sentence(text):
    text = text.replace('<query>', '')
    text = text.replace('</query>', '')
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()
    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]
    return ' '.join(tokens)

In [ ]:
qcorpus_list[1]

{'chatNoirUrl': 'https://chatnoir.eu/cache?uuid=25e04d49-8df7-58c9-8fae-c5bd54a070ba&index=cw12&raw&plain',
 'contents': 'By insisting on upholding affirmative action in college admissions, they ensure that America continues to be an extremely race sensitive society where everything is based on race. For Asian-Americans, I say take heart. Competition only makes you stronger. And who cares about the Ivy League, their mission is never to educate the best. Their main mission is to live off their “old money” prestige by rubber stamping the diploma of the offsprings of the rich and powerful, before ushering them into pre-ordained careers on Wall Street and Washington DC to further enhance the schools reputation and future coffer. A few “colored folks” get to come along for the ride just so they could claim equality. At the end of the day, an ivy league degree only matters if you want to a degree of the idle rich, i.e. a liberal arts degree, before going into law, wall street, politics or ac

In [ ]:
preprocess_sentence(qcorpus_list[1]['contents'])

'insisting upholding affirmative action college admissions ensure america continues extremely race sensitive society everything based race asian-americans say take heart competition makes stronger cares ivy league mission never educate best main mission live “ old money ” prestige rubber stamping diploma offsprings rich powerful ushering pre-ordained careers wall street washington dc enhance schools reputation future coffer “ colored folks ” get come along ride could claim equality end day ivy league degree matters want degree idle rich e liberal arts degree going law wall street politics academia careers depend heavily connections ivy league important'

In [ ]:
# Downloading the model for similarity comparison.
from simcse import SimCSE
model = SimCSE("princeton-nlp/sup-simcse-roberta-large")

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/664 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

In [ ]:
topics_embeddings = {}
for id_, q_ in zip(touche_topics['number'], input_queries):
    embedding_val = model.encode(preprocess_sentence(q_));
    topics_embeddings[id_] = embedding_val

In [ ]:
corpus_embeddings = {}
for topic_id , doc_ids in solution_dict.items():
        for doc_id in doc_ids:
            if doc_id not in corpus_embeddings:
                d_ = doc_id
                doc_content = ''
                for el_ in qcorpus_list:
                    if el_['id'] == d_:
                        doc_content = preprocess_sentence( el_['contents'] )
                        break
                corpus_embeddings[doc_id] = model.encode(doc_content);
        print('Topic '+ str(topic_id) + ' embedding conversion is completed!')

In [ ]:
# storing the finally generated '.pkl' dictionary file.
import pickle

with open('/content/drive/MyDrive/touche-2022-prototyping/document_embeddings.pickle', 'wb') as handle:
    pickle.dump(corpus_embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/drive/MyDrive/touche-2022-prototyping/topic_embeddings.pickle', 'wb') as handle:
    pickle.dump(topics_embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# For calculating the cosine similarity scores.
# ranking code: https://github.com/jacklxc/ParagraphJointModel/blob/main/SentVecAbstractRetriaval.py
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
all_similarities = {}
for claim_id, claim_embedding in tqdm(topics_embeddings.items()):
    this_claim = {}
    for abstract_id, abstract_embedding in corpus_embeddings.items():
        claim_similarity = cosine_similarity(claim_embedding.reshape(1, -1), abstract_embedding.reshape(1, -1))
        this_claim[abstract_id] = claim_similarity
    all_similarities[claim_id] = this_claim

100%|██████████| 3/3 [00:09<00:00,  3.25s/it]


In [ ]:
import numpy as np

In [ ]:
ordered_corpus = {}
for claim_id, claim_similarities in tqdm(all_similarities.items()):
    corpus_ids = []
    max_similarity = []
    for abstract_id, similarity in claim_similarities.items():
        corpus_ids.append(abstract_id)
        max_similarity.append(np.max(similarity))
    corpus_ids = np.array(corpus_ids)
    sorted_order = np.argsort(max_similarity)[::-1]
    ordered_corpus[claim_id] = corpus_ids[sorted_order]

100%|██████████| 3/3 [00:00<00:00, 15.23it/s]


In [ ]:
# getting the output of the sorted out corpus.
ordered_corpus

{1: array(['clueweb12-1214wb-88-29751___2', 'clueweb12-1412wb-36-28142___28',
        'clueweb12-0005wb-10-28805___10', ...,
        'clueweb12-0008wb-25-09896___19', 'clueweb12-0007wb-98-34560___20',
        'clueweb12-0601wb-34-13975___31'], dtype='<U31'),
 2: array(['clueweb12-1804wb-90-15876___2', 'clueweb12-0818wb-38-16958___7',
        'clueweb12-0200wb-97-10905___2', ...,
        'clueweb12-0609wb-58-02331___82',
        'clueweb12-0100tw-23-23985___143', 'clueweb12-1810wb-11-28803___8'],
       dtype='<U31'),
 3: array(['clueweb12-0000wb-60-28210___2', 'clueweb12-0111wb-59-06595___42',
        'clueweb12-0200wb-56-01430___11', ...,
        'clueweb12-0609wb-58-02331___83', 'clueweb12-1118wb-68-13795___9',
        'clueweb12-1402wb-32-34254___5'], dtype='<U31')}

In [ ]:
# getting the retrieved items in list form
retrieved_corpus = {ID:v[:1375].tolist() for ID, v in ordered_corpus.items()}

In [ ]:
# Final output of the lists.
print(len(retrieved_corpus[1]), len(retrieved_corpus[2]), len(retrieved_corpus[3]))

1375 1375 1375


In [ ]:
# getting the retrieved items in list form
retrieved_corpus_baseline = {ID:v[:1375] for ID, v in solution_dict.items()}

In [ ]:
# Final output of the lists.
print(len(retrieved_corpus_baseline[1]), len(retrieved_corpus_baseline[2]), len(retrieved_corpus_baseline[3]))

1375 1375 1375


In [51]:
# getting merged solution dict calculating the average recall/coverage metric.
metric_dict_dense = {}
metric_dict_bm25 = {}
for (id_a, dense_list), (id_b, bm25_list) in zip(retrieved_corpus.items(), retrieved_corpus_baseline.items()):
    # 1.
    d_list = []
    for h_ in dense_list:
        d_= h_.split('___')[0]
        if d_ not in d_list:
            d_list.append(d_)
    metric_dict_dense[id_a] = d_list
    # 2.
    d_list = []
    for h_ in bm25_list:
        d_= h_.split('___')[0]
        if d_ not in d_list:
            d_list.append(d_)
    metric_dict_bm25[id_b] = d_list


In [53]:
i = 0
for x, y in metric_dict_dense.items():
    print(x, len(y))
    if i >= 10:
        break
    i = i+1
print('\n')
i = 0
for x, y in metric_dict_bm25.items():
    print(x, len(y))
    if i >= 10:
        break
    i = i+1

1 431
2 647
3 485


1 593
2 630
3 423


In [54]:
# loading the baseline qrel files.
new_rel_2021 = pd.read_csv('/content/drive/MyDrive/touche-2022-prototyping/touche_ground_truth.csv')
new_rel_2021.head()

,qid,no,doc,rel
0,1,0,clueweb12-0001wb-05-12311,0
1,1,0,clueweb12-1811wb-62-08424,1
2,1,0,clueweb12-1811wb-62-08423,1
3,1,0,clueweb12-1217wb-47-14048,0
4,1,0,clueweb12-1811wb-62-08425,1


In [55]:
from collections import defaultdict
ground_truth_dict = defaultdict(list)
rel0_truth_dict = defaultdict(list)
rel1_truth_dict = defaultdict(list)
rel2_truth_dict = defaultdict(list)

for i_, d_, x_ in zip(new_rel_2021['qid'], new_rel_2021['doc'], new_rel_2021['rel']):
    i_ = int(i_)
    d_ = str(d_)    
    if int(x_) > 0 and i_ in [1,2,3]:
        ground_truth_dict[i_].append(d_)
    if int(x_) == 0 and i_ in [1,2,3]:
        rel0_truth_dict[i_].append(d_)
    if int(x_) == 1 and i_ in [1,2,3]:
        rel1_truth_dict[i_].append(d_)
    if int(x_) == 2 and i_ in [1,2,3]:
        rel2_truth_dict[i_].append(d_)

In [62]:
# the final dictionaries for basic metric evaluation and analysis.
# Average percentage common, Hit-once and Hit-all metric basic definition.
hit_one = 0
hit_all = 0
total = 3
per_comm_avg = 0

for id_i, doc_i in ground_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_dense.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit_one += 1
            if doc_j.issuperset(doc_i):
                hit_all += 1
            per_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Hit one: {round(hit_one / total, 4)}')
print(f'Hit all: {round(hit_all / total, 4)}')
print(f'Average common ratio: {round(per_comm_avg / total, 4)}')

hit0_one = 0
hit0_all = 0
per0_comm_avg = 0

for id_i, doc_i in rel0_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_dense.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit0_one += 1
            if doc_j.issuperset(doc_i):
                hit0_all += 1
            per0_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Zero Relevance, Hit one: {round(hit0_one / total, 4)}')
print(f'Zero Relevance, Hit all: {round(hit0_all / total, 4)}')
print(f'Zero Relevance, Average common ratio: {round(per0_comm_avg / total, 4)}')

hit1_one = 0
hit1_all = 0
per1_comm_avg = 0

for id_i, doc_i in rel1_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_dense.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit1_one += 1
            if doc_j.issuperset(doc_i):
                hit1_all += 1
            per1_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'One Relevance, Hit one: {round(hit1_one / total, 4)}')
print(f'One Relevance, Hit all: {round(hit1_all / total, 4)}')
print(f'One Relevance, Average common ratio: {round(per1_comm_avg / total, 4)}')

hit2_one = 0
hit2_all = 0
per2_comm_avg = 0

for id_i, doc_i in rel2_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_dense.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit2_one += 1
            if doc_j.issuperset(doc_i):
                hit2_all += 1
            per2_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Two Relevance, Hit one: {round(hit2_one / total, 4)}')
print(f'Two Relevance, Hit all: {round(hit2_all / total, 4)}')
print(f'Two Relevance, Average common ratio: {round(per2_comm_avg / total, 4)}')

Hit one: 1.0
Hit all: 0.0
Average common ratio: 0.7565
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.0
Zero Relevance, Average common ratio: 0.5861
One Relevance, Hit one: 1.0
One Relevance, Hit all: 0.0
One Relevance, Average common ratio: 0.6684
Two Relevance, Hit one: 1.0
Two Relevance, Hit all: 0.0
Two Relevance, Average common ratio: 0.844


In [63]:
# the final dictionaries for basic metric evaluation and analysis.
# Average percentage common, Hit-once and Hit-all metric basic definition.
hit_one = 0
hit_all = 0
total = 3
per_comm_avg = 0

for id_i, doc_i in ground_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_bm25.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit_one += 1
            if doc_j.issuperset(doc_i):
                hit_all += 1
            per_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Hit one: {round(hit_one / total, 4)}')
print(f'Hit all: {round(hit_all / total, 4)}')
print(f'Average common ratio: {round(per_comm_avg / total, 4)}')

hit0_one = 0
hit0_all = 0
per0_comm_avg = 0

for id_i, doc_i in rel0_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_bm25.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit0_one += 1
            if doc_j.issuperset(doc_i):
                hit0_all += 1
            per0_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Zero Relevance, Hit one: {round(hit0_one / total, 4)}')
print(f'Zero Relevance, Hit all: {round(hit0_all / total, 4)}')
print(f'Zero Relevance, Average common ratio: {round(per0_comm_avg / total, 4)}')

hit1_one = 0
hit1_all = 0
per1_comm_avg = 0

for id_i, doc_i in rel1_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_bm25.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit1_one += 1
            if doc_j.issuperset(doc_i):
                hit1_all += 1
            per1_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'One Relevance, Hit one: {round(hit1_one / total, 4)}')
print(f'One Relevance, Hit all: {round(hit1_all / total, 4)}')
print(f'One Relevance, Average common ratio: {round(per1_comm_avg / total, 4)}')

hit2_one = 0
hit2_all = 0
per2_comm_avg = 0

for id_i, doc_i in rel2_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in metric_dict_bm25.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit2_one += 1
            if doc_j.issuperset(doc_i):
                hit2_all += 1
            per2_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Two Relevance, Hit one: {round(hit2_one / total, 4)}')
print(f'Two Relevance, Hit all: {round(hit2_all / total, 4)}')
print(f'Two Relevance, Average common ratio: {round(per2_comm_avg / total, 4)}')

Hit one: 1.0
Hit all: 0.0
Average common ratio: 0.7879
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.3333
Zero Relevance, Average common ratio: 0.7778
One Relevance, Hit one: 1.0
One Relevance, Hit all: 0.0
One Relevance, Average common ratio: 0.6987
Two Relevance, Hit one: 1.0
Two Relevance, Hit all: 0.0
Two Relevance, Average common ratio: 0.8917
